<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/project/CookingInstanceModality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir -p local_modules/cooking_inst_mod

In [0]:
%%writefile local_modules/cooking_inst_mod/__init__.py

def get_precision_recall_f1(p, r): 
  index_pred = []
  index_real = []

  prediction_meals = []
  real_meals = []

  #p = flat_pred
  #r = test_label

  #Costruzione delle liste di indici: una di pasti predetti e una di pasti reali

  for i in range(0, len(p)-1):
    if p[i] == 1:
      index_pred.append(i)
      if (p[i+1] == 0): 
        if(len(index_pred)>5):        
          prediction_meals.append(index_pred)
        index_pred = []
    if r[i] == 1:
      index_real.append(i)
      if (r[i+1] == 0): 
        real_meals.append(index_real)
        index_real = []

  #Ricerca delle intersezioni

  intersection = [] #contiene 1 se per un pasto reale j è stata già segnata un'intersezione 
  tp = 0
  fp = 0

  for i in range(0, len(real_meals)): #inizializzazione a 0 per l'array di intersezioni
    intersection.append(0)

  for i in range(0, len(prediction_meals)): #per ogni pasto predetto
    flag_found = 0 #per tenere conto se l'intersezione è già stata trovata all'interno dei pasti reali
    count_int = 0 #tiene conto del numero di intersezioni che un pasto predetto ha con i pasti reali (per il caso in cui un pasto pred intersechi due pasti reali)

    for j in range(0, len(real_meals)):# per ogni pasto reale    
      flag_visited = 0 #tiene conto se un pasto di prediction meals è già stato analizzato (per il caso in cui due pasti pred intersechino un unico pasto reale)

      for x in prediction_meals[i]: #per ogni minuto di pasto controllo se esiste un'intersezione dentro real meals

        if x in real_meals[j]: #se c'è l'intersezione        

          if intersection[j]==0: #ciò significa che non è mai stata trovata un intersezione per il pasto reale j-esimo

            if count_int == 0: #controlla che non siano ancora state trovate intersezioni  
              count_int = count_int + 1
              intersection[j] = 1
              flag_visited = 1 #imposta il flag a uno per indicare che il pasto reale è stato già analizzato e per non contare più di una volta il tp trovato
              flag_found = 1 #imposta il flag a uno per indicare che il pasto predetto è stato trovato nei pasti reali
              tp=tp+1

            elif count_int > 0: #caso in cui un pasto pred interseca due pasti reali e quindi un tp è già stato trovato in precedenza (produce un falso positivo e un vero positivo)           
              intersection[j] = 1
              flag_visited = 1 #imposta il flag a uno per indicare che il pasto reale è stato già analizzato e per non contare più di una volta il tp e il fp trovati
              flag_found = 1            
              fp=fp+1
              tp=tp+1

          elif flag_visited == 0: #caso in cui l'intersezione per il pasto reale j-esimo è già stata trovata ma tale pasto non è ancora stato analizzato con il pasto predetto i-esimo
                                  #risolve il caso in cui più pasti pred intersecano un unico pasto reale
            flag_found = 1
            flag_visited = 1
    #end for j

    if flag_found == 0: #se l'intersezione nn è stata trovata nei pasti reali il pasto predetto è falso positivo
      fp=fp+1
    
    
  #Ricerca falsi negativi

  fn = 0

  for i in range(0, len(real_meals)): #per ogni pasto reale
    j=0
    flag_found = 0
    while j in range(0, len(prediction_meals)) and flag_found==0: # per ogni pasto predetto finchè non viene trovato tra questi l'intersezione con il pasto i-esimo

      for x in real_meals[i]: #per ogni minuto di pasto controllo se esiste un'intersezione dentro prediction_meals

        if x in prediction_meals[j]: #se c'è l'intersezione        
          flag_found = 1 #imposta il flag a uno per indicare che il pasto reale è stato trovato nei pasti predetti

      j = j + 1  
    #end while j

    if flag_found == 0: #se l'intersezione del pasto reale non è stata trovata nei pasti predetti siamo in corrispondenza di un falso negativo
      fn=fn+1


  print("N° pasti reali:", len(real_meals))
  print("N° pasti predetti:", len(prediction_meals))

  print("TP:", tp)
  print("FP:", fp)
  print("FN:", fn)

  somma = tp+fp
  precision = (tp/(tp+fp))*100
  TPR = (tp/(tp+fn))*100
  F1 = 2*((precision*TPR)/(precision+TPR))
  print("Recall: %.2f %%" % TPR)
  print("Precision: %.2f %%" % precision)
  print("F1 score: %.2f %%" % F1)

Writing local_modules/cooking_inst_mod/__init__.py
